# 9a. Crime Statistics

This lab is for Thursday October 31st.

In [1]:
%cd ../../mp248-course

/home/user/mp248-course


In [2]:
#import cdat
from cdat import cDat

Crime database library.

Looking for vicpd.geojson in directory ./Data/
 modify the cdatfile and cdatloc variables in cdat.py appropriately
 if the file can not be found.

Complete records in cDat, incomplete records in bDat.



---

## Ex.1

Construct a dictionary whose keys are the crime types, and whose values are lists (in descending order) of the five dates with the most crimes of that type. Only include crimes where there are **at least** five dates with six or more crimes. Print your dictionary with the **pprint** command.

**Hint:** There are only 16 crime types that meet the above criterion.

In [3]:
import collections as co
import pprint as pp
import operator as op
import calendar as cal

In [4]:
pp.pprint([cDat[x] for x in range(1)])

[{'ID': 760527874,
  'address': '1200 Block JOHNSON ST',
  'case_number': (16, 22918),
  'city': 'victoria',
  'coords': (-123.348998693722, 48.4255680718449),
  'cre_dt': datetime.datetime(2016, 5, 26, 9, 46, 43),
  'inc_dt': datetime.datetime(2016, 5, 25, 1, 29),
  'type': ('Disorder', 'CAUSE A DISTURBANCE'),
  'upd_dt': datetime.datetime(2016, 6, 23, 19, 40, 33)}]


In [5]:
types = {c['type'] for c in cDat}
# pp.pprint(types)

In [6]:
a = [3,'fsd,',20,'lol',90]

In [7]:
d = a[::2]

In [8]:
d

[3, 20, 90]

In [9]:
di = dict()
for typ in types:
    ctdict = co.defaultdict(int)
    for c in cDat:
        if c['type'] == typ:
            ctdict[c['inc_dt'].date()] += 1
    sorted_ctdict = sorted(list(ctdict.items()), key = op.itemgetter(1))[-1:-6:-1]
    if sorted_ctdict[-1][1] > 5:
        di[typ] = sorted_ctdict 

In [10]:
pp.pprint(di)
print("length =",len(di))

{('Assault', 'ASSAULT-COMMON OR TRESPASS'): [(datetime.date(2011, 10, 29), 9),
                                             (datetime.date(2012, 2, 26), 9),
                                             (datetime.date(2011, 9, 2), 7),
                                             (datetime.date(2011, 5, 25), 7),
                                             (datetime.date(2011, 3, 26), 7)],
 ('Disorder', 'CAUSE A DISTURBANCE'): [(datetime.date(2014, 7, 6), 13),
                                       (datetime.date(2015, 4, 20), 11),
                                       (datetime.date(2015, 4, 18), 10),
                                       (datetime.date(2011, 7, 3), 10),
                                       (datetime.date(2014, 10, 5), 10)],
 ('Liquor', 'LIQUOR-INTOX IN PUBLIC PLACE'): [(datetime.date(2011, 7, 1), 44),
                                              (datetime.date(2013, 7, 1), 33),
                                              (datetime.date(2015, 7, 1), 28),
        

---

## Ex. 2

Write a function that uses [spherical polar coordinates](https://en.wikipedia.org/wiki/Spherical_coordinate_system) to convert the (latitude,longitude) coordinate pairs into 3-dimensional Euclidean coordinates.  Compute the average location (in 3-dimensional space) of all the crime types in the **cdat.cDat** variable.  Invert your spherical polar coordinates, creating a dictionary indexed by crime types, returning the (latitude, longitude) pair of the average location. Print your final dictionary using **pp.pprint**. 

**Note:** if you do not recall how latitude and longitude relate to spherical polar coordinates, read the above Wikipedia article carefully. 

In [29]:
import numpy as np
import math as ma

In [97]:
def polar_to_cartesian(lat,long):
    '''
    Convert spherical polar coordinates to 3D Euclidean coordinates
    In this case, r = 1
    '''
    # x = rcos(lat)cos(long)
    # y = rcos(lat)sin(long)
    # z = rcos(lat)
    # lat == theta, phi == long
    # First: Convert degree to radians for numpy
    x = np.cos(ma.radians(lat))*np.cos(ma.radians(long))
    y = np.cos(ma.radians(lat))*np.sin(ma.radians(long))
    z = np.cos(ma.radians(lat))
    return(x, y, z)

In [99]:
def cartesian_to_polar(x_coords, y_coords, z_coords):
    '''
    Convert cartesian coordinates to spherical polar coordinates
    '''
    # lat == theta, phi = long
    r = 1
    lat = ma.degrees(np.arccos(z_coords/r))
    long = ma.degrees(np.arctan(y_coords/x_coords))
    if long < 0:
        long = 180 + long
    return(lat, long)
    

In [105]:
print("(125, 55) ->",polar_to_cartesian(125, 55))
print("->",cartesian_to_polar(-0.3289899283371657, -0.46984631039295427, -0.5735764363510462))

(125, 55) -> (-0.3289899283371657, -0.46984631039295427, -0.5735764363510462)
-> (125.00000000000001, 55.0)


In [60]:
# x = rcos(lat)cos(long)
# y = rcos(lat)sin(long)
# z = rsin(lat)
dict2 = dict()
## For each type of crimes, take (lat, long), convert to Cartesian
## Then take the avg of each x,y,z coords by sum/(#num of crimes)
## Convert back to polar. Return a dictionary of avg polar.
for typ in types:
    sum_x = 0
    sum_y = 0
    sum_z = 0
    total_crime_count = 0
    ctdict = co.defaultdict(int)
    for c in cDat:
        if c['type'] == typ:
            ctdict[c['coords']] += 1
    for i in ctdict.items():
        lat1 = i[0][0]
        long1 = i[0][1]
        crime_count = i[1]
        x, y, z = polar_to_cartesian(lat1, long1)
        #print(i[0])
        #print(x,y,z)
        total_crime_count += crime_count
        sum_x += x
        sum_y += y
        sum_z += z 
    #print(sum_x,sum_y, sum_z)
    #pp.pprint(total_crime_count)
    avg_coords = (sum_x/ total_crime_count, sum_y/total_crime_count, sum_z/total_crime_count)
    dict2[typ] = cartesian_to_polar(avg_coords[0], avg_coords[1],avg_coords[2])


In [61]:
pp.pprint(dict2)

{('Alarm', 'FALSE ALARMS'): (146.63330806412364, 48.43305635617424),
 ('Arson', 'ARSON-DISREGARD FOR HUMAN LIFE'): (146.635657172566,
                                               48.43261481297608),
 ('Arson', 'ARSON-PROPERTY'): (146.63165070607982, 48.43207584459001),
 ('Assault', 'ASSAULT AGAINST POLICE OFFICER'): (146.63279563908142,
                                                 48.430129202907885),
 ('Assault', 'ASSAULT-COMMON OR TRESPASS'): (146.63025337150535,
                                             48.431310491960396),
 ('Assault', 'ASSAULTS-OTHER'): (146.63155020544772, 48.429763093631365),
 ('Assault with Deadly Weapon', 'ASSAULT OT P/O-W/WPN OR CBH'): (146.6336414532833,
                                                                 48.43281681031936),
 ('Assault with Deadly Weapon', 'ASSAULT POLICE-W/WEAPON OR CBH'): (146.63299200828453,
                                                                    48.43052768682063),
 ('Assault with Deadly Weapon', 'ASSAUL

---

## Ex.3. Central Saanich vs. Victoria

What differences do you notice between the crimes reported from Central Saanich, and the ones reported from Victoria? i.e. are the crime types reported in Central Saanich occuring at different frequencies than in Victoria?  Do they occur for a different range of dates?  What appears special about the Central Saanich records?